In [ ]:
%matplotlib inline

# Extract car parking spaces from mongoDB, export as SVG for nesting, stitch nested SVGs 🚗 🚗 🚗

This notebook extracts geometries (areas, like polygons of parking spaces) from a mongoDB, then exports all areas in an svg file for nesting with SVGNest. In the end, the SVG bins are stitched back together.

Created on:  2016-10-28  
Last update: 2017-03-30  
Contact: michael.szell@gmail.com (Michael Szell)

## Preliminaries

### Parameters

In [ ]:
cityname = "vienna"

mode = "car" # do car here. bike is another file

pathdatain = 'output/'+cityname+'/'+mode+'in/'
pathdataout = 'output/'+cityname+'/'+mode+'out/'

### Imports

In [ ]:
from __future__ import unicode_literals
import sys
import csv
import os
import math
from random import shuffle, choice, uniform
import random
import pprint
pp = pprint.PrettyPrinter(indent=4)
from collections import defaultdict
import time
import datetime
import numpy as np
from numpy import *
from scipy import stats
import pyprind
import itertools
import logging
from collections import OrderedDict

import json
from xml.dom import minidom
from shapely.geometry import mapping, shape, LineString, LinearRing, Polygon, MultiPolygon
import shapely
import shapely.ops as ops
from shapely import affinity
from functools import partial
import pyproj
Projection = pyproj.Proj("+proj=merc +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +units=m +no_defs")
from scipy.ndimage.interpolation import rotate
from scipy.spatial import ConvexHull

import pymongo
from pymongo import MongoClient

# plotting stuff
import matplotlib.pyplot as plt

### DB connection

In [ ]:
client = MongoClient()
db = client[cityname+'_derived']
ways = db['ways']
cursor = ways.find({"$and": [{"properties.amenity.amenity": "parking"}, {"geometry.type": "Polygon"}, {"properties_derived.area": { "$gte": 12 }}]}).sort("properties_derived.area",-1)
numparkingareas = cursor.count()
print("There are " + str(numparkingareas) + " " + mode + " parking spaces in " + cityname)

### Functions

In [ ]:
def coordinatesToSVGString(coo, xoffset = 0, yoffset = 0, idname = "", classname = "", rot = 0, centroidlatlon = [0,0]):
    svgstring = "\n  <polygon"
    if idname:
        svgstring += " id=\""+idname+"\""
    if classname:
        svgstring += " class=\""+classname+"\""
    svgstring += " points=\""
    strxylist = [str(coo[i][0]+xoffset)+","+str(coo[i][1]+yoffset) for i in range(coo.shape[0])]
    for s in strxylist:
        svgstring += s+" "
    svgstring += "\""
    svgstring += " moovel_rot=\""+str(rot)+"\"" # pseudo-namespace, because svgnest strips namespace info. http://stackoverflow.com/questions/15532371/do-svg-docs-support-custom-data-attributes
    centroid = [Polygon(coo).centroid.x, Polygon(coo).centroid.y]
    svgstring += " moovel_centroid=\""+str(centroid[0]+xoffset)+","+str(centroid[1]+yoffset)+"\""
    svgstring += " moovel_centroidlatlon=\""+str(centroidlatlon[0])+","+str(centroidlatlon[1])+"\""
    svgstring += "/>"
    return svgstring

def drawPolygon(poly, title=""): # poly is a shapely Polygon
    x, y = poly.exterior.xy
    fig = plt.figure(figsize=(4,4), dpi=90)
    ax = fig.add_subplot(111)
    ax.set_title(title)
    ax.plot(x, y)
    
def getLargestSubPolygon(multipoly): # multipoly is a shapely polygon or multipolygon
    # if its a polygon, do nothing, else give largest subpolygon
    if not (isinstance(multipoly, shapely.geometry.multipolygon.MultiPolygon)):
        return multipoly
    else:
        a = 0
        j = 0
        for i in range(len(multipoly)):
            if multipoly[i].area > a:
                j = i
                a = multipoly[i].area
        return multipoly[j]
    
def getSmallestSubPolygon(multipoly): # multipoly is a shapely polygon or multipolygon
    # if its a polygon, do nothing, else give largest subpolygon
    if not (isinstance(multipoly, shapely.geometry.multipolygon.MultiPolygon)):
        return multipoly
    else:
        a = float("inf")
        j = 0
        for i in range(len(multipoly)):
            if multipoly[i].area < a:
                j = i
                a = multipoly[i].area
        return multipoly[j]

def getTwoLargestSubPolygons(multipoly): # multipoly is a shapely polygon or multipolygon
    # if its a polygon, do nothing, else give two largest subpolygon
    if not (isinstance(multipoly, shapely.geometry.multipolygon.MultiPolygon)):
        return multipoly
    else:
        a = [multipoly[i].area for i in range(len(multipoly))]
        sortorder = sorted(range(len(a)), key=lambda k: a[k], reverse=True) # http://stackoverflow.com/questions/7851077/how-to-return-index-of-a-sorted-list
        return MultiPolygon([ multipoly[i] for i in sortorder[0:2] ])

def rotationToSmallestWidthRecursive(poly, maxdepth = 3, w = float("inf"), rot = 0, rotdelta = 10, depth = 1): # poly is a shapely polygon
    # unit: degrees
    # returns the angle the polygon needs to be rotated to be at minimum width
    # Note: Is not guaranteed to converge to the global minimum
    # Requires import numpy as np, from shapely import affinity
    if depth <= maxdepth:
        for theta in np.arange(rot-rotdelta*9, rot+rotdelta*9, rotdelta):
            temp = affinity.rotate(poly, theta, origin='centroid')
            x, y = temp.exterior.coords.xy
            temp = np.array([[x[i],y[i]] for i in range(len(x))])
            objectwidth = max(temp[:, 0])-min(temp[:, 0])
            if objectwidth < w:
                w = objectwidth
                rot = theta
        return rotationToSmallestWidthRecursive(poly, maxdepth, w, rot, rotdelta/10, depth+1)
    else:
        return rot
    
def getCoordinatesFromSVG(filepath, reversexdir = False, b = 1): # The SVG needs to have polygons with ids, embedded in gs
    doc = minidom.parse(filepath)  # parseString also exists
    path_strings = [path.getAttribute('points') for path
                    in doc.getElementsByTagName('polygon')]
    id_strings = [path.getAttribute('id') for path
                    in doc.getElementsByTagName('polygon')]
    class_strings = [path.getAttribute('class') for path
                    in doc.getElementsByTagName('polygon')]
    g_strings = [path.getAttribute('transform') for path
                in doc.getElementsByTagName('g')]
    rot_strings = [path.getAttribute('moovel_rot') for path
                    in doc.getElementsByTagName('polygon')]
    centroidlatlon_strings = [path.getAttribute('moovel_centroidlatlon') for path
                    in doc.getElementsByTagName('polygon')]
    doc.unlink()
    data = dict()
    numbins = 0
    for i,path in enumerate(path_strings):
        if class_strings[i] == "bin":
            numbins += 1
        if numbins == b:
            path = path.split()
            coo = []
            for temp in path:
                p = temp.split(",")
                try:
                    trans = g_strings[i] # looks like this: "translate(484.1119359029915 -1573.8819930603422) rotate(0)"
                    trans = trans.split()
                    trans = [float(trans[0][10:]), float(trans[1][0:-1])] # gives [484.1119359029915,-1573.8819930603422]
                except:
                    trans = [0,0]
                if reversexdir:
                    coo.append([-(float(p[0])+trans[0]), float(p[1])+trans[1]])
                else:
                    coo.append([float(p[0])+trans[0], float(p[1])+trans[1]])
            data[id_strings[i]] = dict()
            data[id_strings[i]]["coordinates"] = coo
            data[id_strings[i]]["rot"] = rot_strings[i]
            data[id_strings[i]]["class"] = class_strings[i]
            data[id_strings[i]]["centroidlatlon"] = centroidlatlon_strings[i].split(",")
            
        elif numbins > b:
            break
    return data

## Get parking spaces for multiple SVG bins

### Features
1. add buffer around big tiles ✔️
2. give more bin space for small tiles on top of bins ✔️
3. pre-select tiles from DB and exhaust them fully (and uniquely!) ✔️
4. do not order big tiles by size, but more randomly ✔️
5. add id as id, and more properties ✔️

In [ ]:
parameters = {"car": {"berlin":{"maxbigparts":180, "maxbins":16}, 
                      "newyork":{"maxbigparts":29, "maxbins":7},
                      "stuttgart":{"maxbigparts":30, "maxbins":3},
                      "amsterdam":{"maxbigparts":20, "maxbins":2},
                      "portland":{"maxbigparts":70, "maxbins":4},
                      "vienna":{"maxbigparts":30, "maxbins":5},
                      "losangeles":{"maxbigparts":50, "maxbins":11},
                      "sanfrancisco":{"maxbigparts":16, "maxbins":2},
                      "boston":{"maxbigparts":8, "maxbins":1},
                      "budapest":{"maxbigparts":26,"maxbins":5},
                      "hongkong":{"maxbigparts":8,"maxbins":1},
                      "beijing":{"maxbigparts":3,"maxbins":2}, 
                      "helsinki":{"maxbigparts":30,"maxbins":7},
                      "copenhagen":{"maxbigparts":20,"maxbins":3},
                      "london":{"maxbigparts":100,"maxbins":20},
                      "chicago":{"maxbigparts":80, "maxbins":18},
                      "jakarta":{"maxbigparts":6, "maxbins":1},
                      "moscow":{"maxbigparts":90, "maxbins":21},
                      "rome":{"maxbigparts":30, "maxbins":6},
                      "singapore":{"maxbigparts":14, "maxbins":3},
                      "tokyo":{"maxbigparts":35, "maxbins":7},
                      "johannesburg":{"maxbigparts":15, "maxbins":3},
                      "barcelona":{"maxbigparts":5,"maxbins":1}
                     }
             }

# to find out maxbins, make 10 bins or so. If all have same file size, increase until file size gets small. Select maxbins to cover all big parts.
# to find out maxbigparts, set to around maxbins*6. Possibly decrease by looking at bigparts.svg 
maxbins = parameters[mode][cityname]["maxbins"]

In [ ]:
# get parking spaces (ALL in a city)
cursor = ways.find({"$and": [{"properties.amenity.amenity": "parking"}, {"geometry.type": "Polygon"}, {"properties_derived.area": { "$gte": 12 }}]}).sort("properties_derived.area",-1)
random.seed(1)
maxbigparts = parameters[mode][cityname]["maxbigparts"]
scale = 0.6
erectbigparts = True
erectnonbigparts = True
randomrotatenonbigparts = False
binareafactor = 0.83 # This factor ensures that there are slightly more parking spaces than could fit into one bin. We later collect all leftover parking spots from those second bins.
smallvsmedium = 11
buffereps = 5 # should be the same number as the distances between parts in SVGNest
height = 1200
width = 600 - 1.5*buffereps
draw = False # for debugging purposes (drawing all big parts and bins) set this to True
eps = 0.000001

bigbin = Polygon([[0,0], [width,0], [width, maxbins*height], [0, maxbins*height]])
bigbin = bigbin.difference(Polygon([[width/2-eps,-1], [width/2-eps,maxbins*height-2], [width/2+eps,maxbins*height-2], [width/2+eps,-1]]))

# pre-select all parts
idsused = set()
idsnotused = set()
indicesused = set()
alltiles = []
alltileskeys = []
alltilesarea = 0
for i,way in enumerate(cursor):
    npway = np.asarray(way["geometry"]["coordinates"])
    centroidlatlon = [Polygon(npway).centroid.x, Polygon(npway).centroid.y]
    npwayxy = [Projection(npway[i][0], npway[i][1]) for i in range(npway.shape[0])]
    npwayxy = np.asarray([[npwayxy[i][0],-npwayxy[i][1]] for i in range(npway.shape[0])])
    if i < maxbigparts: # big parts
        if erectbigparts:
            rot = rotationToSmallestWidthRecursive(Polygon(npwayxy))
        else:
            rot = 0
    else: # non-big parts
        if erectnonbigparts:
            rot = rotationToSmallestWidthRecursive(Polygon(npwayxy))
        elif randomrotatenonbigparts:
            rot = uniform(10, 350)
        else:
            rot = 0
    if rot:
        temp = affinity.rotate(Polygon(npwayxy), rot, origin='centroid', use_radians=False)
        x, y = temp.exterior.coords.xy
        npwayxy = np.array([[x[i],y[i]] for i in range(len(x))])
    objectwidth = max(npwayxy[:, 0])-min(npwayxy[:, 0])
    npwayxy[:, 0] -= min(npwayxy[:, 0])
    npwayxy[:, 1] -= min(npwayxy[:, 1])
    npwayxy *= scale
    objectwidth *= scale
    if objectwidth < width:
        objectheight = max(npwayxy[:, 1])
        idsnotused.add(way["_id"])
        coo = [[npwayxy[k][0], npwayxy[k][1]] for k in range(npwayxy.shape[0])]
        area = Polygon(coo).buffer(buffereps/2).area
        alltiles.append( { "_id": way["_id"], "width": objectwidth, "height": objectheight, "area": area, "coordinates": coo , "rot": rot, "centroidlatlon": centroidlatlon})
        alltileskeys.append(way["_id"])
        alltilesarea += area
    else:
        print("Object "+str(way["_id"])+" was too wide (" +str(objectwidth)+ " pixel) and was ignored.")

bigpartstodiff = []
partareastaken = []
ypos = -0.5*buffereps
numbigparts = 0
randomizedindices = list(range(maxbigparts))
shuffle(randomizedindices)
# Determine big parts
for i in randomizedindices:
    tile = alltiles[i]
    if ypos >= height*maxbins-1-tile["height"]: # see if this part still fits
        break
    tile["coordinates"] = [[-1*tile["coordinates"][k][0]+width/2+tile["width"]/2, tile["coordinates"][k][1]+ypos] for k in range(np.array(tile["coordinates"]).shape[0])]
    bigpartstodiff.append( tile )
    ypos += tile["height"] + 1*buffereps
        
    indicesused.add(i)
    idsused.add(tile["_id"])
    idsnotused.remove(tile["_id"])
    numbigparts += 1 # increase number of parts in any case
    if draw:
        drawPolygon(Polygon(bigpartstodiff[-1]["coordinates"]), "Big part "+str(numbigparts))

# Export the big parts
svg = "<svg xmlns=\"http://www.w3.org/2000/svg\" version=\"1.1\" width=\""+str(width)+"px\" height=\""+str(height)+"px\">"
for i in range(len(bigpartstodiff)):
    svg += coordinatesToSVGString(np.array(bigpartstodiff[i]["coordinates"]), 0, 0, str(bigpartstodiff[i]["_id"]), "tile", bigpartstodiff[i]["rot"], bigpartstodiff[i]["centroidlatlon"])
svg += "\n</svg>"
with open(pathdataout + "bigparts.svg", "w") as f:
    f.write(svg)
            
# Clip bins into batches and diff the big parts
for j in range(numbigparts):
    bigbin = getLargestSubPolygon(bigbin.difference(Polygon(bigpartstodiff[j]["coordinates"]).buffer(1.75*buffereps, 1, 2, 2)))
        
# Cut the big part into sub-bins
scissorv = Polygon([[width/2-eps, -1], [width/2-eps, maxbins*height+1], [width/2+eps, maxbins*height+1], [width/2+eps, -1]])
cutbins = [[], []]
bigbin = getTwoLargestSubPolygons(bigbin.difference(scissorv)) # cut in half vertically

for m in range(len(bigbin)): # cut horizontally
    rest = bigbin[m]
    for i in range(maxbins):
        scissorh = Polygon([[-1, (i+1)*height], [width+1, (i+1)*height], [width+1, (i+1)*height+eps], [-1, (i+1)*height+eps]])
        temp = rest
        temp = getTwoLargestSubPolygons(temp.difference(scissorh))
        cutbins[m].append(getSmallestSubPolygon(temp).buffer(0.75*buffereps, 1, 2, 2))
        rest = getLargestSubPolygon(temp)
        if draw:
            drawPolygon(cutbins[m][-1], "Bin "+str(i)+",  m:"+str(m))
            
# Fill with small parts
remainingtiles = len(idsnotused)
if remainingtiles > 0:
    randomizedindices_medium = list(range(round(remainingtiles/smallvsmedium)+numbigparts))
    randomizedindices_medium = list(set(randomizedindices_medium) - indicesused)
    shuffle(randomizedindices_medium)
else:
    randomizedindices_medium = []
randomizedindices_small = set(list(range(remainingtiles+numbigparts))) - set(randomizedindices_medium)
randomizedindices_small = list(randomizedindices_small - indicesused)
shuffle(randomizedindices_small)

imedium = 0
ismall = 0
for numbin in range(maxbins):
    for mirrored in [0,1]:
        if mirrored:
            m = -1
        else:
            m = 1
        binarea = binareafactor*cutbins[mirrored][numbin].area
        binbound = np.array(cutbins[mirrored][numbin].exterior.coords)
        binbound = np.asarray([[-m*(binbound[i,0]-min(binbound[:,0])/2),binbound[i,1]-min(binbound[:,1])] for i in range(binbound.shape[0])])
        xpos = 0
        ypos = height+1 # start placing the elements below the bin
        yextent = 0
        svg = "<svg xmlns=\"http://www.w3.org/2000/svg\" version=\"1.1\" width=\""+str(width)+"px\" height=\""+str(height)+"px\">"
        svg += coordinatesToSVGString(binbound, 0, 0, str(numbin), "bin")
        totalarea = 0
        
        for j in range(len(idsnotused)):
            if len(idsnotused) == 0:
                    break
            try:
                i = randomizedindices_medium[imedium]
                imedium += 1
            except: # no more medium tiles left
                i = randomizedindices_small[ismall]
                ismall += 1
            tile = alltiles[i]
            if tile["width"] <= width:
                if xpos + tile["width"] + 1 <= width: # there is space in this row
                    xdelta = m*(xpos+1)
                    ydelta = ypos
                else: # new row
                    xdelta = 0
                    ypos += yextent
                    yextent = 0
                    ydelta = ypos
                    xpos = 0
                svg += coordinatesToSVGString(np.array([[m*tile["coordinates"][k][0], tile["coordinates"][k][1]] for k in range(np.array(tile["coordinates"]).shape[0])]), xdelta, ydelta, str(tile["_id"]), "tile", tile["rot"], tile["centroidlatlon"])
                yextent = max([yextent, tile["height"]])
                xpos += tile["width"]+1
                idsused.add(tile["_id"])
                idsnotused.remove(tile["_id"])
                totalarea += tile["area"]
                if totalarea > binarea:
                    break
            else:
                print("Object "+str(way["_id"])+" was too wide (" +str(max(npwayxy[:, 0]))+ " pixel) and could not be placed.")
            for kk in range(smallvsmedium):
                if len(idsnotused) == 0:
                    break
                try:
                    i = randomizedindices_small[ismall]
                    ismall += 1
                except:
                    i = randomizedindices_medium[imedium]
                    imedium += 1
                tile = alltiles[i]
                if tile["width"] <= width:
                    if xpos + tile["width"] + 1 <= width: # there is space in this row
                        xdelta = m*(xpos+1)
                        ydelta = ypos
                    else: # new row
                        xdelta = 0
                        ypos += yextent
                        yextent = 0
                        ydelta = ypos
                        xpos = 0
                    svg += coordinatesToSVGString(np.array([[m*tile["coordinates"][k][0], tile["coordinates"][k][1]] for k in range(np.array(tile["coordinates"]).shape[0])]), xdelta, ydelta, str(tile["_id"]), "tile", tile["rot"], tile["centroidlatlon"])
                    yextent = max([yextent, tile["height"]])
                    xpos += tile["width"]+1
                    idsused.add(tile["_id"])
                    idsnotused.remove(tile["_id"])
                    totalarea += tile["area"]
                    if totalarea > binarea:
                        break
                else:
                    print("Object "+str(way["_id"])+" was too wide (" +str(max(npwayxy[:, 0]))+ " pixel) and could not be placed.")
            if totalarea > binarea:
                break
        svg += "\n</svg>"

        if mirrored:
            with open(pathdatain + cityname + mode + "parking"+ str(numbin).zfill(3)  +"min.svg", "w") as f:
                f.write(svg)
        else:   
            with open(pathdatain + cityname + mode + "parking"+ str(numbin).zfill(3)  +"in.svg", "w") as f:
                f.write(svg)
                
print("First export done. " + str(len(idsnotused))+" tiles were not used.")

This has generated bigparts.svg in {{pathdataout}}, and {{maxbins}}*2 files in {{pathdatain}}. Use SVGNest on these latter files. Move the files returned from SVGNest (Download folder) into {{pathdataout}}.

## After SVGNest was executed a 1st time, collect all leftover parking spots

In [ ]:
# Collect leftover tiles
idsinsecondbins = set()
for i in range(maxbins):
    for mirrored in [0,1]:
        if mirrored:
            m = -1
            tiles = getCoordinatesFromSVG(pathdataout + str(i).zfill(3) + "m.svg", False, 2)
        else:
            m = 1
            tiles = getCoordinatesFromSVG(pathdataout + str(i).zfill(3) + ".svg", False, 2)
        for key in tiles:
            if tiles[key]["class"] == "tile":
                idsinsecondbins.add(int(key))

idsnotusedtotal = idsnotused | idsinsecondbins

print(str(len(idsnotused))+" tiles were not used first.")
print(str(len(idsinsecondbins))+" tiles were in second bins.")
print(str(len(idsnotusedtotal))+" tiles were not used in total. Packing them into an extra bin...")


# Calculate needed area
leftovertilesarea = 0
for j in idsnotusedtotal:
    i = int(np.where(np.array(alltileskeys) == j)[0])
    tile = alltiles[i]
    leftovertilesarea += tile["area"]
    
numidsnotusedtotal = len(idsnotusedtotal)

# Make an extra bin pair for the leftover tiles
bigbin = Polygon([[0,0], [width,0], [width, height], [0, height]])
bigbinarea = bigbin.area

# change the big bin area according to the leftover tiles area
heightx = round(height * leftovertilesarea/bigbinarea)
bigbin = Polygon([[0,0], [width,0], [width, heightx], [0, heightx]])

scissorv = Polygon([[width/2-eps, -1], [width/2-eps, maxbins*heightx+1], [width/2+eps, maxbins*heightx+1], [width/2+eps, -1]])
bigbin = getTwoLargestSubPolygons(bigbin.difference(scissorv)) # cut in half vertically
numbin = 0
cutbins = [[], []]
randomizedindices = list(idsnotusedtotal)
shuffle(randomizedindices)

for m in range(len(bigbin)): # cut horizontally
    rest = bigbin[m]
    for i in range(maxbins):
        scissorh = Polygon([[-1, (i+1)*heightx], [width+1, (i+1)*heightx], [width+1, (i+1)*heightx+eps], [-1, (i+1)*heightx+eps]])
        temp = rest
        temp = getTwoLargestSubPolygons(temp.difference(scissorh))
        cutbins[m].append(getSmallestSubPolygon(temp).buffer(0.75*buffereps, 1, 2, 2))
        rest = getLargestSubPolygon(temp)           

for mirrored in [0,1]:
    if mirrored:
        m = -1
    else:
        m = 1
    binbound = np.array(cutbins[mirrored][numbin].exterior.coords)
    binbound = np.asarray([[(binbound[i,0]-min(binbound[:,0])/2),binbound[i,1]-min(binbound[:,1])] for i in range(binbound.shape[0])])
    xpos = 0
    ypos = heightx+buffereps # start placing the elements below the bin
    yextent = 0
    svg = "<svg xmlns=\"http://www.w3.org/2000/svg\" version=\"1.1\" width=\""+str(width)+"px\" height=\""+str(heightx)+"px\">"
    svg += coordinatesToSVGString(binbound, 0, 0, str(maxbins+1), "bin")
    while len(randomizedindices):
        j = randomizedindices[0]
        if len(idsnotusedtotal) <= int(numidsnotusedtotal - (mirrored+1)*numidsnotusedtotal/2): # put half the tiles here, half in the other bin
            break
        i = int(np.where(np.array(alltileskeys) == j)[0])
        tile = alltiles[i]
        if tile["width"] <= width:
            if xpos + tile["width"] + 1 <= width: # there is space in this row
                xdelta = m*(xpos+1)
                ydelta = ypos
            else: # new row
                xdelta = 0
                ypos += yextent
                yextent = 0
                ydelta = ypos
                xpos = 0
            svg += coordinatesToSVGString(np.array([[m*tile["coordinates"][k][0], tile["coordinates"][k][1]] for k in range(np.array(tile["coordinates"]).shape[0])]), xdelta, ydelta, str(tile["_id"]), "tile", tile["rot"], tile["centroidlatlon"])
            yextent = max([yextent, tile["height"]])
            xpos += tile["width"]+1
            idsnotusedtotal.remove(tile["_id"])
            k = int(np.where(np.array(randomizedindices) == j)[0])
            del randomizedindices[k]
        else:
            print("Object "+str(way["_id"])+" was too wide (" +str(max(npwayxy[:, 0]))+ " pixel) and could not be placed.")
    svg += "\n</svg>"

    # Export
    if mirrored:
        with open(pathdatain + cityname + mode + "parking"+ "extra" +"min.svg", "w") as f:
            f.write(svg)
    else:   
        with open(pathdatain + cityname + mode + "parking"+ "extra" +"in.svg", "w") as f:
            f.write(svg)

This has generated 2 files in {{pathdatain}}, called *extra*. Use SVGNest on these files. Move the files returned from SVGNest into {{pathdataout}}, and rename them to have the number {{maxbins}}+1. (so, 000m.svg becomes, for example, 007m.svg, if the last filename was 006m.svg)

## After SVGNest was executed a 2nd time, stitch back together all the parts

In [ ]:
allmirrored = True 
# There is a bug: For some cities, all non-big parts are mirrored. Not yet known why, so we are using this hack. Yeah..
# In this case just set allmirrored to True and execute this cell again (no need to run everything again).

swap = False
swapm = True
deltashifts = False
# Another issue, solved by swap and swapm: last and second last bins (before the extra bin) are swapped on one (or both) side. This is most likely due to very large big parts.
# deltashifts: sometimes the big parts are slightly shifted, fixed by deltashifts


# read SVG
numbins = maxbins+1
alltilesfinal = []

# Big parts
bigpartscoo = getCoordinatesFromSVG(pathdataout + "bigparts.svg", not allmirrored, 0)
for k in bigpartscoo:
    bigpartscoo[k]["coordinates"] = [[bigpartscoo[k]["coordinates"][i][0]+width-int(allmirrored)*width, bigpartscoo[k]["coordinates"][i][1]] for i in range(len(bigpartscoo[k]["coordinates"]))]
alltilesfinal.append(bigpartscoo)
ypos = 0
xpos = 0

ypos -= 0.75*buffereps
xpos = width/2

                        
# Rest
for j in range(numbins):
    for mirrored in [0,1]:
        i = j
        if mirrored:
            m = -1
            if allmirrored:
                tiles = getCoordinatesFromSVG(pathdataout + str(i).zfill(3) + ".svg", mirrored)
            else:
                if swapm and numbins >= 3:
                    if j == numbins-3:
                        i = numbins-2
                    if j == numbins-2:
                        i = numbins-3
                tiles = getCoordinatesFromSVG(pathdataout + str(i).zfill(3) + "m.svg", mirrored)
        else:
            m = 1
            if allmirrored:
                tiles = getCoordinatesFromSVG(pathdataout + str(i).zfill(3) + "m.svg", mirrored)
            else:
                if swap and numbins >= 3:
                    if j == numbins-3:
                        i = numbins-2
                    if j == numbins-2:
                        i = numbins-3
                tiles = getCoordinatesFromSVG(pathdataout + str(i).zfill(3) + ".svg", mirrored)
        minx = float("inf")
        maxx = 0
        for key in tiles:
            if tiles[key]["class"] == "tile":
                npwayxy = np.array(tiles[key]["coordinates"])
                minx = min(minx, mirrored*width/2+min([npwayxy[k,0] for k in range(npwayxy.shape[0])]))
                maxx = max(maxx, mirrored*width/2+max([npwayxy[k,0] for k in range(npwayxy.shape[0])]))
        maxx = width/2-maxx
        if mirrored:
            if allmirrored:
                delta = minx
            else:
                delta = -minx
        else:
            if allmirrored:
                delta = maxx
            else:
                delta = -maxx
        if not deltashifts:
            delta = 0
        for key in tiles:
            if tiles[key]["class"] == "tile":
                npwayxy = np.array(tiles[key]["coordinates"])
                if allmirrored:
                    npwayxy = [[npwayxy[k,0]+xpos-m*width/2-delta, npwayxy[k,1]+ypos] for k in range(npwayxy.shape[0])]
                else:
                    npwayxy = [[npwayxy[k,0]+xpos-delta, npwayxy[k,1]+ypos] for k in range(npwayxy.shape[0])]
                alltilesfinal.append({key: {"coordinates": npwayxy, "rot": tiles[key]["rot"], "centroidlatlon": tiles[key]["centroidlatlon"]}})
    ypos += height

if allmirrored: # need to mirror all back
    for i in range(len(alltilesfinal)):
        tile = alltilesfinal[i]
        for key in tile:
            npwayxy = np.array(tile[key]["coordinates"])
            tile[key]["coordinates"] = [[width-npwayxy[k,0], npwayxy[k,1]] for k in range(npwayxy.shape[0])]
            alltilesfinal[i] = tile
    
# Export
svg = "<svg xmlns=\"http://www.w3.org/2000/svg\" version=\"1.1\" width=\""+str(width)+"px\" height=\""+str(height*(numbins-1)+heightx)+"px\">"
for j, tile in enumerate(alltilesfinal):
    for i in tile:
        svg += coordinatesToSVGString(np.array([[tile[i]["coordinates"][k][0], tile[i]["coordinates"][k][1]] for k in range(np.array(tile[i]["coordinates"]).shape[0])]), 0, 0, i, "", tile[i]["rot"], tile[i]["centroidlatlon"])
svg += "\n</svg>"
with open(pathdataout + "all.svg", "w") as f:
    f.write(svg)

The result is all.svg in {{pathdataout}}.